In [ ]:
""" Categorize CAT inspection into Trades according to keywords contained in the assessment finding text """

import login
import numpy as np
import pandas as pd
import sqlalchemy as sa
import mysql.connector

try:
    engine = sa.create_engine('mysql+mysqlconnector://{user}:{pw}@{hostname}/flbwmass'\
    .format(user=login.username, pw=login.password, hostname=login.mysql_host))
except mysql.connector.Error as e:
    print(e)
finally:
    engine.dispose()

qry = "select cat_mass_inspection_items.*, period, contract, wo_type from cat_mass_inspection_items left join cat_mass_inspections using (report_serial_num)"
df = pd.read_sql(qry, con=engine)
df['aspect'] = np.where(df['subaspect'].isnull(), df['aspect'], df['subaspect'])
df['item'] = df.item_description.str[:3]
df = df.loc[(df['grade'] != 'N') & (df['aspect'] != 'JP'), ['report_serial_num', 'period', 'contract', 'wo_type', 'aspect', 'item', 'grade', 'grade_description']].copy()

In [ ]:
brickwork = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Bb]rick|[Yy][\W_]*[Tt]ong'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Bb]rick')))]
brickwork['trade_assignment'] = 'Brickwork'
carpentry = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Dd]oor[\W_]*[Hh]inge|[dD]oor[\W_]*[Ll]ock|[Bb]arrel[\W_]*[Bb]olt'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Aa]rchitrave|[Ww]ooden[\W_]*[Dd]oor|[Tt]imb[er][er][\W_]*[Dd]oor|[Mm]itre[|W_]*[Jj]oint')))]
carpentry['trade_assignment'] = 'Carpentry'
screeding = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Ss]creed(ing)*'))]
screeding['trade_assignment'] = 'Cement Sand screeding'
hotline = df.loc[df.grade_description.str.contains('[Hh]otline')]
hotline['trade_assignment'] = 'Customer Services-Hotline'
notice = df.loc[df.grade_description.str.contains('([Cc]ommencement|[Cc]ompletion)[\W_]*[Dd]ate')]
notice['trade_assignment'] = 'Customer Services-Notice of works'
worker = df.loc[df.grade_description.str.contains('[Uu]niform|[Ll][Rr][Ii][Ss][\W_]*[Cc]ard')]
worker['trade_assignment'] = 'Customer Services-Worker'
environment = df.loc[df.grade_description.str.contains('[Ff]loor[\W_]*[Dd]rain')]
environment['trade_assignment'] = 'GSM-Environment'
housekeeping = df.loc[df.grade_description.str.contains('[Hh]ouse[\W_]*[Kk]eep(ing)*')]
housekeeping['trade_assignment'] = 'GSM-Housekeeping'
procedure = df.loc[df.grade_description.str.contains('[Pp]rocedure|A[.]*S[.]*A|[Aa]ppointment')]
procedure['trade_assignment'] = 'GSM-Procedure'
tested_worker = df.loc[df.grade_description.str.contains('[Tt]rade')]
tested_worker['trade_assignment'] = 'GSM-Trade tested worker'
burgler_grille = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Gg]rille'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Gg]rille')))]
burgler_grille['trade_assignment'] = 'Metal Works-burglar grille'
metal_door = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Mm]etal[\W_]*[Dd]oor|[Cc]ollapsible[\W_]*[Gg]ate'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Mm]etal[\W_]*[Dd]oor|[Cc]ollapsible[\W_]*[Gg]ate')))]
metal_door['trade_assignment'] = 'Metal Works-metal door & gate'
window_louvre = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Ww]indow|[Ll]ouvre'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Ww]indow|[Ll]ouvre')))]
window_louvre['trade_assignment'] = 'Metal Works-window & louvre'
barrier = df.loc[df.grade_description.str.contains('[Bb]arrier|[Ww]arning|[Ss]ign|[Ff]ence')]
barrier['trade_assignment'] = 'Safety-Barrier & warning sign'
scaffolding = df.loc[df.grade_description.str.contains('[Ff]orm[\W_]*5|[Ss]caffold(ing)*|[Ll]adder|[Mm]etal[\W_]*[Pp]latform|[Ww]orking[\W_]*[Pp]latform|[Bb]amboo|[Ee][Nn]131|[Gg]ondola|[Pp]lanking|[Nn]et')]
scaffolding['trade_assignment'] = 'Safety-Scaffolding & Ladder'
electricity = df.loc[df.grade_description.str.contains('[Ww][.]*[Rr][.]*1|[Ee]lectricity[\W_]*[Ss]upply|[Gg]as|[Ww]ire|[Ll]oose[\W_]*[Pp]articles|[Ss]afety[\W_]*[Hh]elmet|[Cc]igarette')]
electricity['trade_assignment'] = 'Safety-Temporary Electricity & PPE'
sanitary = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Bb]asin|[Cc]istern|[Cc]loset|[Ww]ater[\W_]*[Pp]ipe|[Ss]ink'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Bb]asin|[Cc]istern|[Cc]loset|[Ww]ater[\W_]*[Pp]ipe|[Pp]ipe[\W_]*[Bb]racket|[Ss]ink')))]
sanitary['trade_assignment'] = 'Sanitary Fittings'
fire_collar = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Ff]ire[\W_]*[Cc]ollar'))]
fire_collar['trade_assignment'] = 'Sanitary Fittings-Fire collar'
bench = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('([Ss]ink|[Cc]ooking)[\W_]*[Bb]ench'))]
bench['trade_assignment'] = 'Sanitary Fittings-Sink & cooking bench'
spalling = df.loc[(df.aspect == 'M') & (df.grade_description.str.contains('[Uu]nicell[\W_]*40|[Ss]palling[\W_]*[Rr]epair[\W_][Mm]ortar'))]
spalling['trade_assignment'] = 'Spalling Repair'
finishes = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Rr]epair(ed)*[\W_]*[Pp]atch(es)*'))]
finishes['trade_assignment'] = 'Spalling Repair-Finishes'
saw_cut = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Bb]reakout|[Cc]learance'))]
saw_cut['trade_assignment'] = 'Spalling Repair-Saw cut'
touchup = df.loc[((df.aspect == 'M') & (df.grade_description.str.contains('[Pp]aint'))) | ((df.aspect == 'W') & (df.grade_description.str.contains('[Ss]ealer|[Pp]aint')))]
touchup['trade_assignment'] = 'Touch-up'
finishing = df.loc[(df.aspect == 'M') & (df.grade_description.str.contains('[Cc]ement|[Tt]ile'))]
finishing['trade_assignment'] = 'Wall & Floor Finishing'
tile = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Tt]ile'))]
tile['trade_assignment'] = 'Wall & Floor Finishing-Finishes'
plastering = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Pp]lastering'))]
plastering['trade_assignment'] = 'Wall & Floor Finishing-Plastering'
td = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Kk]eys'))]
td['trade_assignment'] = 'Wall & Floor Finishing-T/D, Saw Cut'
waterproofing = df.loc[(df.aspect == 'M') & (df.grade_description.str.contains('[Ww]aterproofing[\W_]*[Mm]aterial'))]
waterproofing['trade_assignment'] = 'Waterproofing works'
angle_fillet = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Aa]ngle[\W_]*[Ff]illet|[Ss]kirting'))]
angle_fillet['trade_assignment'] = 'Waterproofing works-Angle fillet'
membrane = df.loc[(df.aspect == 'W') & (df.grade_description.str.contains('[Ww]aterproofing[\W_]*[Mm]embrane'))]
membrane['trade_assignment'] = 'Waterproofing works-Finishes'

classified = pd.concat([brickwork, carpentry, screeding, hotline, notice, worker, environment, housekeeping, procedure, tested_worker, burgler_grille, metal_door, window_louvre, barrier, scaffolding, electricity, sanitary, fire_collar, bench, spalling, finishes, saw_cut, touchup, finishing, tile, plastering, td, waterproofing, angle_fillet, membrane, ])

In [ ]:
unclassified = df.loc[df.index.difference(classified.index)].copy()
unclassified['trade_assignment'] = 'Unclassified'

df.sort_index().to_csv('./raw.csv', encoding='utf-8')
pd.concat([classified, unclassified]).sort_index().to_csv('./output/classified-findings.csv', encoding='utf-8')